In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext watermark
%watermark

Last updated: 2023-11-23T06:20:26.094231+05:30

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.15.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
# Either you can store the  OpenAI key in the “OPENAI_API_KEY” environment variable.
# or pass it here as below from a config.ini
import configparser
workingFolder='C:\\Users\\jfrancis\\OneDrive - GalaxE. Solutions, Inc\\GalaxE D Drive\\AI Journey\\Gen AI\\'
# Read the configuration file
config = configparser.ConfigParser()
config.read(workingFolder+'\\config.ini')
OPENAI_API_KEY=config.get('General','OPENAI_API_KEY')
ACTIVELOOP_TOKEN=config.get('General','ACTIVELOOP_TOKEN')
ACTIVELOOP_ORG_ID=config.get('General','ACTIVELOOP_ORG_ID')
HUGGINGFACEHUB_API_TOKEN=config.get('General','HUGGINGFACEHUB_API_TOKEN')
GOOGLE_API_KEY=config.get('General','GOOGLE_API_KEY')
GOOGLE_CSE_ID=config.get('General','GOOGLE_CSE_ID')
COHERE_API_KEY=config.get('General','COHERE_API_KEY')

In [15]:
# Get the token from Hugginface/Active loop website before this. Now we are taking from the config.ini
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN
os.environ["ACTIVELOOP_TOKEN"] = ACTIVELOOP_TOKEN
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = ACTIVELOOP_ORG_ID

In [4]:
import langchain
import tqdm
import requests
import typing
%watermark --iversions 

tqdm     : 4.66.1
langchain: 0.0.336
requests : 2.31.0



First, let’s download financial reports from Amazon and load them into pages:

In [5]:
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI 
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PagedPDFSplitter
import requests
import tqdm
from typing import List

# financial reports of amamzon, but can be replaced by any URLs of pdfs
urls = ['https://s2.q4cdn.com/299287126/files/doc_financials/Q1_2018_-_8-K_Press_Release_FILED.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/Q2_2018_Earnings_Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Q318-Amazon-Earnings-Press-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/AMAZON.COM-ANNOUNCES-FOURTH-QUARTER-SALES-UP-20-TO-$72.4-BILLION.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/Q119_Amazon_Earnings_Press_Release_FINAL.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Amazon-Q2-2019-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Q3-2019-Amazon-Financial-Results.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_news/archive/Amazon-Q4-2019-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/Q1/AMZN-Q1-2020-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/q2/Q2-2020-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2020/q4/Amazon-Q4-2020-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q1/Amazon-Q1-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q2/AMZN-Q2-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q3/Q3-2021-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2021/q4/business_and_financial_update.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q1/Q1-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q2/Q2-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q3/Q3-2022-Amazon-Earnings-Release.pdf',
        'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/Q4-2022-Amazon-Earnings-Release.pdf'
        ]

def load_reports(urls: List[str]) -> List[str]:
    """ Load pages from a list of urls"""
    pages = []

    for url in tqdm.tqdm(urls):
        r = requests.get(url)
        path = url.split('/')[-1]
        with open(path, 'wb') as f:
            f.write(r.content)
        loader = PagedPDFSplitter(path)
        local_pages = loader.load_and_split()
        pages.extend(local_pages)
    return pages

pages = load_reports(urls)

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:42<00:00,  5.40s/it]


We now use the Text Splitter Util to split documents into pages.

In [16]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

Now use a open source hugging face embedding as the openAI one could be costly (due to the volume of data)

In [18]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

Load it to active loop vector store

In [19]:
my_activeloop_dataset_name = "amazon_earnings"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding=embeddings)
db.add_documents(texts)

Your Deep Lake dataset has been successfully created!


Creating 414 embeddings in 1 batches of size 414:: 100%|████████████████████████████████| 1/1 [04:28<00:00, 268.15s/it]

Dataset(path='hub://jfrancis/amazon_earnings', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (414, 1)     str     None   
 metadata     json      (414, 1)     str     None   
 embedding  embedding  (414, 768)  float32   None   
    id        text      (414, 1)     str     None   


['25fc4196-89a5-11ee-93f8-401c83da435e',
 '25fc4197-89a5-11ee-b59d-401c83da435e',
 '25fc4198-89a5-11ee-9de1-401c83da435e',
 '25fc4199-89a5-11ee-8330-401c83da435e',
 '25fc419a-89a5-11ee-b3c7-401c83da435e',
 '25fc419b-89a5-11ee-b4a7-401c83da435e',
 '25fc419c-89a5-11ee-b800-401c83da435e',
 '25fc419d-89a5-11ee-be58-401c83da435e',
 '25fc419e-89a5-11ee-b3fe-401c83da435e',
 '25fc419f-89a5-11ee-9a91-401c83da435e',
 '25fc41a0-89a5-11ee-8c08-401c83da435e',
 '25fc41a1-89a5-11ee-a31b-401c83da435e',
 '25fc41a2-89a5-11ee-bff6-401c83da435e',
 '25fc41a3-89a5-11ee-bda2-401c83da435e',
 '25fc41a4-89a5-11ee-a884-401c83da435e',
 '25fc41a5-89a5-11ee-bea0-401c83da435e',
 '25fc41a6-89a5-11ee-8557-401c83da435e',
 '25fc41a7-89a5-11ee-b598-401c83da435e',
 '25fc41a8-89a5-11ee-ae6f-401c83da435e',
 '25fc41a9-89a5-11ee-885a-401c83da435e',
 '25fc41aa-89a5-11ee-bf28-401c83da435e',
 '25fc41ab-89a5-11ee-80f1-401c83da435e',
 '25fc41ac-89a5-11ee-99b7-401c83da435e',
 '25fc41ad-89a5-11ee-bcc5-401c83da435e',
 '25fc41ae-89a5-

Since the dataset is stored on Activeloop, you can load it later without recomputing embeddings. This is a significant benefit cause it would save you time and computational resources. LangChain has a wrapper around Deep Lake, allowing you to use it as a Vector Store.

In [9]:
#db = DeepLake(dataset_path="hub://jfrancis/amazon_earnings", read_only=True, embedding=embeddings)

In [37]:
# Retrieval queue from activeloop using OpenAI LLM
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type='stuff', retriever=db.as_retriever())

### How to answer questions using ML?

The state-of-the-art method of question answering is by leveraging LangChain and Activeloop Deep Lake. You can store your documents in the unified, streamable format, and connect it to LangChain to run a question-answering model such as GPT or BERT. These language models are able to understand the context of the question and generate a more accurate answer. You may also implement techniques like data augmentation, document retrieval, and summarization to enhance the system performance.

Let’s now explore Amazon’s revenue change pre- and post-pandemic.

In [38]:
qa.run("What is the revenue in 2018 Q3?")
# The net sales for Q3 2021 was $110.8 billion, with a 15% increase to $127.1 billion in Q3 2022.

' Amazon.com announced third quarter sales up 29% to $56.6 billion in 2018.'

In [39]:
qa.run("Combine total revenue in 2020?")
#Amazon's total revenue in 2020 was $386,064 million.

' $75,452 million'

In [40]:
qa.run("What is the revenue in 2023 Q1?")
# The financial results for 2023 Q1 are not provided in the given context

" I don't know."

Obviously, the question answering works only on the given data, that is limited to 2022. To answer more current questions, we should connect LangChain to agents like GoogleSearchAPIWrapper.